In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [24]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=307
p_folder="../../plots/run_"+str(r_n)+"/"

In [25]:
# helper functions
def choose_bar(df,layer, strip): #works
    df=df[df["strip"]==strip]
    df=df[df["layer"]==layer]
    return df

def import_data(calibration_folder, data_folder, run_number, is_it_pulsed=False): # works for both pulsed and not-pulsed
    
    pedestal_df=pd.read_csv(calibration_folder+"pedestal_fixed.csv", sep=',')
    mip_df=pd.read_csv(calibration_folder+"mip.csv", sep=',')
    
    if is_it_pulsed==True:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+"_pulse.csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True) # we never need these
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True) # unreliable guys, but we might need them in the future
    else:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+".csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True)
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True)
        
    return pedestal_df,mip_df,run_df

In [26]:
p,m,r=import_data(c_folder,d_folder, 307, is_it_pulsed=True)

In [27]:
p

,layer,strip,end,pedestal_end0,std_dev_end0,pedestal_end1,std_dev_end1
0,1,0,0,760,9.674627,840,11.046368
1,1,1,0,760,14.564194,760,18.856047
2,1,2,0,728,10.387670,824,10.888322
3,1,3,0,728,11.938451,824,13.267025
4,1,4,0,824,12.376215,760,11.554757
...,...,...,...,...,...,...,...
187,19,7,0,760,11.239116,744,11.614080
188,19,8,0,792,9.901413,808,10.536791
189,19,9,0,760,8.508676,824,9.375246
190,19,10,0,728,9.138480,824,9.243012


In [ ]:
def select_bars_with_data(run_df, pedestal_df=None, mip_df=None, subtract_pedestal=False, convert_mip=False):
    data_df=[]
    
    layers=np.arange(1,20)
    strips=np.arange(0,12)
    
    for layer in layers:
        for strip in strips:
            df_slice=choose_bar(df,layer,strip)
            pedestal_slice=choose_bar(pedestals,layer,strip)
            mip=choose_bar(mips,layer,strip)
            if not pedestal_slice.empty:
                df_slice=df_slice[df_slice["adc_sum_end0"]>1.2*pedestal_slice.iloc[0,-2]] # there could be a switch whether we want to
                df_slice=df_slice[df_slice["adc_sum_end1"]>1.2*pedestal_slice.iloc[1,-2]] # require both ends to register the hit

                df_slice.loc[:,"adc_sum_end0"]-=pedestal_slice.iloc[0,-2] # subtracting pedestals
                df_slice.loc[:,"adc_sum_end1"]-=pedestal_slice.iloc[1,-2]

                df_slice.loc[:,"adc_sum_end0"]*=(4.66/mip.iloc[0,-1]) # converting to energy
                df_slice.loc[:,"adc_sum_end1"]*=(4.66/mip.iloc[1,-1])
                
            confirmed_data.extend(df_slice.values.tolist())
    confirmed_df=pd.DataFrame(confirmed_data, columns=['event', 'adc_sum_end0', 'layer', 'strip', 'adc_sum_end1'])
    events_left=confirmed_df.event.unique()
    print("Initial pedestal-based selection performed.")
    print("Events left: "+str(len(events_left)))
    print("\n")
    return confirmed_df